# Wishes data visualisation

This notebook is work in progress.

In [ ]:
import pandas as pd

In [ ]:
df_perm = pd.read_csv("./datasets/wishes_permanent.csv")
df_lim = pd.read_csv("./datasets/wishes_character_event.csv")

In [ ]:
def count_sequence_lengths(_df, quality):
    sequence_i = 0
    df = _df.copy()
    
    def label_sequence(row, quality):
        nonlocal sequence_i
        
        if (row["quality"] == quality):
            sequence_i = sequence_i + 1
            return sequence_i - 1
        else:
            return sequence_i

    df["sequence"] = df.apply(lambda row: label_sequence(row, quality), axis=1)
    return df.groupby("sequence").size()

In [ ]:
print(count_sequence_lengths(df_perm, 5))